In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Julio 2024
# ==========================================================
# Modelo lineal multivariante: multicolinealidad

# Importando librerias:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from numpy.linalg import det, cond

# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/salarios.csv'

# Cargar los datos en un DataFrame
datos = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(datos.head())

In [ ]:
# Definir las variables independientes y dependientes
X = datos[['educacion','edad', 'antiguedad_ejecutivo', 'educacion_grado', 'antiguedad', 'educacion_posgrado', 'valor_empresa', 'beneficios_empresa',  'ventas_empresa']]
y = datos['salario']

# Añadir una constante a las variables independientes
X = sm.add_constant(X)

# Ajustar el modelo de regresión lineal
model = sm.OLS(y, X).fit()

# Resumen del modelo
print(model.summary())

In [ ]:
# *** Evaluando multicolinealidad: determinante de la matriz X ***
# Calcular X'X
XTX = np.dot(X.T, X)
# Determinante de X'X
det_XTX = det(XTX)
print(f"Determinante de X'X: {det_XTX}")

In [ ]:
# *** Evaluando multicolinealidad: Numero de condicion de Belsley ***
norm_x = X.values
for i, name in enumerate(X):
    if name == "const":
        continue
    norm_x[:, i] = X[name] / np.linalg.norm(X[name])
norm_xtx = np.dot(norm_x.T, norm_x)
eigs = np.linalg.eigvals(norm_xtx)
numero_de_condicion_de_Belsley = np.sqrt(eigs.max() / eigs.min())
print(numero_de_condicion_de_Belsley)

In [ ]:
# *** Evaluando multicolinealidad: VIF ***
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF para cada variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

In [ ]:
# Eliminar las variable con VIFs más altos:
X_reducido = X.drop(columns=['valor_empresa','beneficios_empresa','ventas_empresa'])

# Ajustar el modelo de regresión lineal con las variables reducidas
modelo_reducido = sm.OLS(datos['salario'], X_reducido).fit()
print(modelo_reducido.summary())


In [ ]:
# *** Evaluando multicolinealidad: Numero de condicion de Belsley ***
norm_x = X_reducido.values
for i, name in enumerate(X_reducido):
    if name == "const":
        continue
    norm_x[:, i] = X_reducido[name] / np.linalg.norm(X_reducido[name])
norm_xtx = np.dot(norm_x.T, norm_x)
eigs = np.linalg.eigvals(norm_xtx)
numero_de_condicion_de_Belsley = np.sqrt(eigs.max() / eigs.min())
print(numero_de_condicion_de_Belsley)

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Estandarizar las variables independientes
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.drop(columns=['const']))

# Aplicar PCA
pca = PCA(n_components=3)  # Número de componentes principales deseados
X_pca = pca.fit_transform(X_scaled)

# Convertir a DataFrame y añadir constante
X_pca = sm.add_constant(pd.DataFrame(X_pca, columns=['CP1', 'CP2', 'CP3']))

# Ajustar el modelo de regresión lineal con las componentes principales
modelo_pca = sm.OLS(datos['salario'], X_pca).fit()
print(modelo_pca.summary())

In [ ]:
# *** Evaluando multicolinealidad: VIF ***
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF para cada variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X_pca.columns
vif_data["VIF"] = [variance_inflation_factor(X_pca.values, i) for i in range(X_pca.shape[1])]
print(vif_data)

In [ ]:
# *** Evaluando multicolinealidad: Numero de condicion de Belsley ***
norm_x = X_pca.values
for i, name in enumerate(X_pca):
    if name == "const":
        continue
    norm_x[:, i] = X_pca[name] / np.linalg.norm(X_pca[name])
norm_xtx = np.dot(norm_x.T, norm_x)
eigs = np.linalg.eigvals(norm_xtx)
numero_de_condicion_de_Belsley = np.sqrt(eigs.max() / eigs.min())
print(numero_de_condicion_de_Belsley)